In [316]:
import tensorflow as tf
import numpy as numpy
import os
import logging


tf.__version__

'2.10.0'

In [317]:

IMG_HEIGHT = 180
IMG_WIDTH = 180


# Define variables
NUM_CLASSES = 2
EPOCHS=10
BATCH_SIZE = 2

DATA_DIR = "D:/Custom Training/train"
DATA_DIR


'D:/Custom Training/train'

In [318]:

def normalize_img(image):
    
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.

def normalize_img_and_label(image, label):    
    return normalize_img(image), label




In [319]:
def parse_image(filename):
    parts = tf.strings.split(filename, os.path.sep)
    label = parts[-2]  # Extract label from the second-to-last part of the path
    one_hot_label = tf.one_hot(tf.cast(label == 'dogs', tf.int32), NUM_CLASSES)
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    return image, one_hot_label

# Define dataset path
DATA_DIR = "./train"

# List files recursively from the dataset directory
file_list = tf.data.Dataset.list_files(DATA_DIR + '/cats/*.jpg') \
            .concatenate(tf.data.Dataset.list_files(DATA_DIR + '/dogs/*.jpg'))

# Map parsing function to the dataset
dataset = file_list.map(parse_image)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

output_directory = "./output"


In [320]:

logging.info('Creating and training model ...')
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(2, activation='sigmoid')  
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])
model.fit(dataset, epochs=10)

logging.info(f'Exporting SavedModel to: {output_directory}')
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
probability_model.save(output_directory)

Epoch 1/10
456/456 [==============================] - 41s 86ms/step - loss: 27.0065 - accuracy: 0.9912
Epoch 2/10
456/456 [==============================] - 34s 75ms/step - loss: 25.5213 - accuracy: 0.9781
Epoch 3/10
456/456 [==============================] - 33s 72ms/step - loss: 1.7341 - accuracy: 0.9693
Epoch 4/10
456/456 [==============================] - 32s 70ms/step - loss: 0.4533 - accuracy: 0.9156
Epoch 5/10
456/456 [==============================] - 34s 74ms/step - loss: 0.4758 - accuracy: 0.9079
Epoch 6/10
456/456 [==============================] - 36s 79ms/step - loss: 0.3632 - accuracy: 0.9068
Epoch 7/10
456/456 [==============================] - 116s 254ms/step - loss: 0.5090 - accuracy: 0.9375
Epoch 8/10
456/456 [==============================] - 34s 75ms/step - loss: 3.9738 - accuracy: 0.8860
Epoch 9/10
456/456 [==============================] - 33s 73ms/step - loss: 0.6033 - accuracy: 0.9287
Epoch 10/10
456/456 [==============================] - 33s 72ms/step - loss: 0

INFO:tensorflow:Assets written to: ./output\assets


INFO:tensorflow:Assets written to: ./output\assets


In [321]:

# Load the saved model
model = tf.saved_model.load('./output/')

# Prepare the input data
img_path = './23.jpg'

img = tf.io.read_file(img_path)
img = tf.image.decode_image(img, channels=3)
img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
img = img / 255.0
img = tf.expand_dims(img, axis=0)

# Make predictions
predictions = model(img)

# Process the predictions
predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
print("Predicted class:", predicted_class)

Predicted class: 1
